## Link to AOAI
First make sure you have created an AML environment and jupyter kernel as in Notebook 1.

Test that you have selected the correct code by running the cell below.

In [1]:
import emoji
print(emoji.emojize('Python is :thumbs_up:'))

Python is 👍


- Create Azure OpenAI Resource [Detailed Instructions](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal)

- Rename credentials_template.env to credentials.env [Detailed Instructions](https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python#retrieve-key-and-endpoint)
- Fill in the credentials.env


In [30]:
# Loads in environement varaibles from credentials.env. 
# Note that if you change the contents of credentials.env you will need to restart your python kernel as well as rerun this cell.
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("credentials.env")

endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_ID")



In [3]:
print("Endpoint: ", endpoint)
print("API Key: ", api_key)
print("Deployment: ", deployment)

Endpoint:  https://bootcamp-vinny.openai.azure.com/
API Key:  c8b1764ad0c546e7b411c182423a704e
Deployment:  Bootcamp


In [ ]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

In [19]:

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful travel assistant. You must answer like Gandalf from Lord of the Rings"},
        {"role": "user", "content": "Can you recomend me a walking holiday in Europe?"},
    ]
)

print(response.choices[0].message.content)

Ah, a walking holiday, you say? If you seek adventure and breathtaking scenery, I would recommend the Camino de Santiago in Spain. It is a renowned pilgrimage route with diverse landscapes, charming towns, and the opportunity to meet fellow travelers. One does not simply walk the Camino without being touched by its magic.


In [ ]:
article = """U.S. intelligence agencies will soon be using a secretive generative artificial intelligence (AI) platform from Microsoft that will let America's spies safely use AI models in the process of analyzing sensitive data.
Microsoft's generative AI model for intelligence agencies aims to get around security issues that stem from large language models' (LLMs) connection to the internet, which typically is used as a resource for training those models. Bloomberg reported that the AI tool is the first major LLM fully separated from the internet.
William Chappell, Microsoft's CTO for strategic missions and technology, told Bloomberg the AI tool was deployed to an "air-gapped" cloud environment isolated from the internet and features a model based on GPT-4 along with supporting tools. The company announced the new product at the AI Expo for National Competitiveness this week.
"This is the first time we've ever had an isolated version — when isolated means it's not connected to the internet — and it's on a special network that's only accessible by the U.S. government," Chappell told Bloomberg."""

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Consider this article: {article} Please create a summary"}
    ]
)

print(response.choices[0].message.content)

**USE YOUR OWN DATA** - From guide https://learn.microsoft.com/en-us/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python 

In [18]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_ID")

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "Summarize the race",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": os.environ["AZURE_AI_SEARCH_ENDPOINT"],
                    "index_name": os.environ["AZURE_AI_SEARCH_INDEX"],
                    "authentication": {
                        "type": "api_key",
                        "key": os.environ["AZURE_AI_SEARCH_API_KEY"],
                    }
                }
            }
        ],   
    },
    temperature=0,
    top_p=1,
    max_tokens=800,
    stop=None,
)

print(completion.model_dump_json(indent=2))

{
  "id": "08ad9ab8-20fe-44ad-ac54-cb9db9f223de",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2022 British Grand Prix was a thrilling and chaotic race, culminating in Carlos Sainz claiming his first Formula 1 victory for Ferrari, with Sergio Perez finishing second and Lewis Hamilton in third place [doc1][doc2]. The race featured intense battles, including Sainz's aggressive restart to claim the lead, Perez's remarkable recovery from P16 to P2, and a fierce fight between Hamilton and Leclerc [doc1][doc2]. The race also saw incidents involving multiple drivers, with some forced into retirement and others facing car damage [doc2][doc3]. Ultimately, Sainz's victory marked a special day for him and Ferrari, setting the stage for the upcoming Austrian Grand Prix at the Red Bull Ring [doc1].\n\n[doc1]: Ferrari's Carlos Sainz has claimed his first Formula 1 victory after a breath-taking contest at the British Grand Prix, the 

**CODE INTERPRETER**

In [34]:
import os
import time
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    )

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("Formula1_2022season_raceResults.csv", "rb"),
  purpose='assistants'
)

# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="You are an AI assistant that answers questions on formula 1 race data",
  model=os.environ.get("AZURE_OPENAI_DEPLOYMENT_ID"),
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)

In [35]:
def askCodeInterpreter(prompt):
    # Create a thread
    thread = client.beta.threads.create()

    # Add a user question to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt # Use the prompt from the function argument
    )

    # Run the thread
    run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id
    )

    # Looping until the run completes or fails
    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

    if run.status == 'completed':
      messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
      print(messages)
    elif run.status == 'requires_action':
      # the assistant requires calling some functions
      # and submit the tool outputs back to the run
      pass
    else:
      print(run.status)

In [23]:
askCodeInterpreter(f"Tell me about the dataset")

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_mm1XpbJqOyrz0i14vCz4Mf9u', assistant_id='asst_OyRl16svxwnuzWBMVEzodTdk', content=[MessageContentText(text=Text(annotations=[], value="The dataset contains the following columns:\n\n1. Track: The name of the race track\n2. Position: The finishing position of the driver in the race\n3. No: The race number of the driver\n4. Driver: The name of the driver\n5. Team: The team the driver is racing for\n6. Starting Grid: The starting position of the driver on the grid\n7. Laps: The number of laps completed by the driver in the race\n8. Time/Retired: The time or status of the driver at the end of the race\n9. Points: The points earned by the driver in the race\n10. +1 Pt: Indicates whether the driver earned an extra point\n11. Fastest Lap: The time of the driver's fastest lap in the race\n\nThis dataset provides information about the finishing positions, times, and other performance indicators of drivers in Formula 1 races."), type='text

In [24]:
askCodeInterpreter(f"which driver received the most points")

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_MEYX6HR3kjuUepTUg9idnERJ', assistant_id='asst_OyRl16svxwnuzWBMVEzodTdk', content=[MessageContentText(text=Text(annotations=[], value='The driver who received the most points is Max Verstappen.'), type='text')], created_at=1718811669, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_moauJT0LwfPcRnPGu5aVzvuL', thread_id='thread_DexdKQtAQEaphTiXVkczhREo'), ThreadMessage(id='msg_AjWTZTl3vCVD6kRjpghEtBlr', assistant_id='asst_OyRl16svxwnuzWBMVEzodTdk', content=[MessageContentText(text=Text(annotations=[], value='It looks like the column names in the file are different than what I expected. The driver\'s name is under the "Driver" column, and the points are under the "Points" column. Let me now calculate which driver received the most points.'), type='text')], created_at=1718811667, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_moauJT0LwfPcRnPGu5aVzvuL', thread_id='thre

In [36]:
askCodeInterpreter(f"draw me a graph showing the points per race gained by each driver across the seassion so I can see their points accumulation and the performance though the season. Give each driver a different colour")

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_rP3UUwawjcJ67hrh8cdoAvis', assistant_id='asst_oolbnczVlVJHxMz4TVR4myrP', content=[MessageContentImageFile(image_file=ImageFile(file_id='assistant-YgJnoMFC12VsdXRD4fIN0veT'), type='image_file'), MessageContentText(text=Text(annotations=[], value="Here is the graph showing the points per race gained by each driver across the season. Each driver is represented by a different color, allowing you to easily compare their points accumulation and performance throughout the season.\n\nThe x-axis represents the race number, while the y-axis represents the points accumulated by each driver. From the graph, you can observe the points accumulation trends for each driver as the season progresses.\n\nIf you have any further questions or if there's anything else you'd like to analyze, feel free to ask!"), type='text')], created_at=1718813509, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_DcMpqvZSvJLrj3WLi0FVP8

In [37]:
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    )

image_data = client.files.content("assistant-YgJnoMFC12VsdXRD4fIN0veT")
image_data_bytes = image_data.read()

with open("./my-image.png", "wb") as file:
    file.write(image_data_bytes)